In [1]:
!pip install dotenv

# For IPYNB Output

In [2]:
import gradio as gr
import os
from datetime import datetime
import json
import traceback
from openai import OpenAI

# Setup OpenAI client
openai = OpenAI(api_key="")# Your API key

system_doc = "You are a Python expert who documents code with clear docstrings and inline comments."

def prompt_for_doc(code):
    return f"Add docstrings and inline comments to this code:\n\n{code}"

def messages_for_doc(code):
    return [
        {"role": "system", "content": system_doc},
        {"role": "user", "content": prompt_for_doc(code)},
    ]

def generate_documentation(code):
    messages = messages_for_doc(code)
    documented_code = ""
    stream = openai.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        stream=True
    )
    for chunk in stream:
        documented_code += chunk.choices[0].delta.content or ""
    return documented_code

def convert_to_notebook(doc_code):
    """Wrap documented Python code into a single Jupyter code cell."""
    return {
        "cells": [
            {
                "cell_type": "code",
                "execution_count": None,
                "metadata": {},
                "outputs": [],
                "source": doc_code.splitlines(keepends=True)
            }
        ],
        "metadata": {
            "kernelspec": {
                "display_name": "Python 3",
                "language": "python",
                "name": "python3"
            },
            "language_info": {
                "name": "python",
                "pygments_lexer": "ipython3"
            }
        },
        "nbformat": 4,
        "nbformat_minor": 5
    }

def process_file(file):
    try:
        # Get filename and extension
        filepath = file.name
        ext = os.path.splitext(filepath)[-1]

        with open(filepath, 'r', encoding='utf-8') as f:
            content = f.read()

        # Extract code depending on file type
        if ext == ".ipynb":
            nb_json = json.loads(content)
            code_cells = [
                cell["source"] for cell in nb_json.get("cells", []) if cell.get("cell_type") == "code"
            ]
            code = "\n\n".join(["".join(cell) for cell in code_cells])
        elif ext == ".py":
            code = content
        else:
            return None, "❌ Unsupported file type. Please upload a .py or .ipynb file."

        # Generate documentation
        documented_code = generate_documentation(code)

        # Convert documented code into notebook format
        nb_data = convert_to_notebook(documented_code)

        # Save notebook
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_path = f"documented_code_{timestamp}.ipynb"
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(nb_data, f, indent=2)

        return output_path, "✅ Successfully documented and exported as .ipynb"

    except Exception as e:
        error_text = traceback.format_exc()
        return None, f"❌ An error occurred:\n\n{error_text}"


# Gradio Interface
ui = gr.Interface(
    fn=process_file,
    inputs=gr.File(label="Upload .py or .ipynb file"),
    outputs=[
        gr.File(label="Download documented .ipynb file"),
        gr.Textbox(label="Process Output / Error Message")
    ],
    title="📓 Auto Code Documenter (.ipynb)",
    description="Upload a Python or Notebook file. It will return a Jupyter Notebook with docstrings and comments using GPT-4o."
)

ui.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c67a9423a138605d29.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# For PY Output

In [3]:
import gradio as gr
import os
from datetime import datetime
import json
import traceback
from openai import OpenAI

# Setup OpenAI client
openai = OpenAI(api_key="") # Your api key

system_doc = "You are a Python expert who documents code with clear docstrings and inline comments."

def prompt_for_doc(code):
    return f"Add docstrings and inline comments to this code:\n\n{code}"

def messages_for_doc(code):
    return [
        {"role": "system", "content": system_doc},
        {"role": "user", "content": prompt_for_doc(code)},
    ]

def generate_documentation(code):
    messages = messages_for_doc(code)
    documented_code = ""
    stream = openai.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        stream=True
    )
    for chunk in stream:
        documented_code += chunk.choices[0].delta.content or ""
    return documented_code

def process_file(file):
    try:
        # Get filename and extension
        filepath = file.name
        ext = os.path.splitext(filepath)[-1]

        # Read file content from the path directly
        with open(filepath, 'r', encoding='utf-8') as f:
            content = f.read()

        # Extract code depending on file type
        if ext == ".ipynb":
            nb_json = json.loads(content)
            code_cells = [
                cell["source"] for cell in nb_json.get("cells", []) if cell.get("cell_type") == "code"
            ]
            code = "\n\n".join(["".join(cell) for cell in code_cells])
        elif ext == ".py":
            code = content
        else:
            return None, "❌ Unsupported file type. Please upload a .py or .ipynb file."

        # Generate documentation
        documented_code = generate_documentation(code)

        # Save to new file
        # Save to new file
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_path = f"documented_code_{timestamp}.py"  # Save in local directory
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(documented_code)


        return output_path, "✅ Successfully documented the code."

    except Exception as e:
        error_text = traceback.format_exc()
        return None, f"❌ An error occurred:\n\n{error_text}"


# Gradio Interface
ui = gr.Interface(
    fn=process_file,
    inputs=gr.File(label="Upload .py or .ipynb file"),
    outputs=[
        gr.File(label="Download documented .py file"),
        gr.Textbox(label="Process Output / Error Message")
    ],
    title="📄 Auto Code Documenter",
    description="Upload a Python or Notebook file. It will return the version with docstrings and comments using GPT-4o."
)

ui.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0d617e975954c55a27.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
